# Explingo Experiment Runner

This notebook:
1. Loads the gold-standard dataset, prepares the metrics functions, and verifies that the metric functions give the maximum score on the gold-standard dataset, and lower scores on less aligned datasets
2. Runs the prompt-design, few-shot, and bootstrap-few-shot experiments on a testing dataset

## Setup
Import necessary libraries and prepare the LLM

**Note: To run these cells, you need a `keys.yaml` file in the top-level Explingo directory with the following line:**
```yaml
openai_api_key: <your_openai_api_key>
```

In [1]:
import pandas as pd

from experiment_runner import ExplingoExperimentRunner
import os
import yaml
import dspy
import metrics
import random

In [2]:
with open(os.path.join("..", "keys.yaml"), "r") as file:
    config = yaml.safe_load(file)
    openai_api_key = config["openai_api_key"]

llm = dspy.OpenAI(model='gpt-4o', api_key=openai_api_key, max_tokens=1000)

Now, we create the main experiment runner object. This object takes in a dataset, and then
1. Splits the dataset into a training dataset and a testing dataset (see notes below)
2. Sets up the evaluation metrics (see notes below). The fluency metric is set up to use sample from the dataset as reference
3. Runs the experiments on the testing dataset

Some examples in the testing datasets include gold-standard narratives; others include only a sample explanation.
- The former makes up the gold-standard dataset used for tuning the evaluation metrics and providing few-shot examples.
- The latter makes up the testing dataset used for evaluation and for bootstrapping few-shot examples

We use the following metrics, all scored on a scale from 0-4:
- Accuracy: the narrative accurately describes the information in the explanation
- Fluency: the narrative is coherent and natural, as compared to the gold-standard explanations. We pass in a small list of sample narratives from the gold-standard dataset to compare against
- Conciseness: the narrative is not too long, as compared to the gold-standard explanations. For now, any narrative that is no longer than the longest gold-standard narrative will score 4
- Completeness: the narrative includes all relevant information from the original explanation 

**Note: You can set `verbose=1` to see the narratives generated, or `verbose=2` to see the explanations, narratives, and rationalizations**

In [6]:
# iterate all datasets in the eval_data folder
runners = {}
total_eval = 0
for dataset in os.listdir(os.path.join("eval_data")):
    runners[dataset] = ExplingoExperimentRunner(llm=llm, openai_api_key=openai_api_key, dataset_filepath = os.path.join("eval_data", dataset), verbose=0)
    total_eval += len(runners[dataset].eval_data)
    
print("Total eval examples:", total_eval)
results = []

eval_data\housing_1.json
Total number of examples: 35
Labeled training examples: 5
Labeled evaluation examples: 15
Unlabeled training examples: 5
Unlabeled evaluation examples: 10
---
eval_data\housing_2.json
Total number of examples: 22
Labeled training examples: 5
Labeled evaluation examples: 7
Unlabeled training examples: 5
Unlabeled evaluation examples: 5
---
eval_data\housing_3.json
Total number of examples: 22
Labeled training examples: 5
Labeled evaluation examples: 8
Unlabeled training examples: 5
Unlabeled evaluation examples: 4
---
eval_data\mushroom_1.json
Total number of examples: 30
Labeled training examples: 5
Labeled evaluation examples: 6
Unlabeled training examples: 5
Unlabeled evaluation examples: 14
---
eval_data\mushroom_2.json
Total number of examples: 30
Labeled training examples: 5
Labeled evaluation examples: 6
Unlabeled training examples: 5
Unlabeled evaluation examples: 14
---
eval_data\pdf_1.json
Total number of examples: 30
Labeled training examples: 5
Label

## Basic prompt design experiment

We begin with basic prompts. With 4 metrics (without completeness), each with a score of 0-2, the maximum score is 8. 

We generate narratives/rationalizations on `max_iters=5` sample explanations, and return the average total score.

In [4]:
# Utilities for cleaner results

def pretty_print(result):
    s = f"Total score: {result[0]}"
    s2 = ", ".join([f"{k}: {v}" for k, v in result[1].items()])
    print(f"{s} ({s2})")
    
def update_results(method, dataset, scores, kwargs):
    result = {"dataset": dataset, "prompt": prompt, "total score": scores[0]}
    result.update(scores[1])
    result.update(kwargs)
    results.append(result)

In [5]:
prompts = ["You are helping users understand an ML model's prediction. Given an explanation and information about the model, "
           "convert the explanation into a human-readable narrative.",
           "You are helping users who do not have experience working with ML understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative. Make your answers sound as natural as possible.",
           "You are helping users understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative. Be sure to explicitly mention all values from the explanation in your response.",
]

for dataset in runners:
    runner = runners[dataset]
    print(f"Dataset: {dataset}")
    for prompt in prompts:
        print(f"Prompt: {prompt}")
        scores = runner.run_basic_prompting_experiment(prompt=prompt)
        update_results("basic_prompting", dataset, scores, {"prompt": prompt})
        pretty_print(scores)
        print("--")
    print("=====")

Dataset: housing_1.json
Prompt: You are helping users understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative.


KeyboardInterrupt: 

## Few-shot experiment

Next, we repeat the experiment with the addition of N few-shot examples from the gold-standard dataset.

In [7]:
for dataset in runners:
    runner = runners[dataset]
    print(f"Dataset: {dataset}")
    for i in [1, 3, 5]:
        if dataset == "housing_1.json" and i in [1, 3]:
            print(f"Skipping {dataset} for n={i}")
            continue
        print(f"Few-shot n: {i}")
        scores = runner.run_few_shot_experiment(n_few_shot=i, prompt=prompts[0])
        update_results("few_shot", dataset, scores, {"n_few_shot": i, "prompt": prompts[0]})
        pretty_print(scores)
        print("--")
    print("=====")

Dataset: housing_1.json
Skipping housing_1.json for n=1
Skipping housing_1.json for n=3
Few-shot n: 5
Total score: 15.24 (accuracy: 3.52, completeness: 4.0, fluency: 3.72, conciseness: 4.0)
--
=====
Dataset: housing_2.json
Few-shot n: 1
Total score: 16.0 (accuracy: 4.0, completeness: 4.0, fluency: 4.0, conciseness: 4.0)
--
Few-shot n: 3
Total score: 16.0 (accuracy: 4.0, completeness: 4.0, fluency: 4.0, conciseness: 4.0)
--
Few-shot n: 5
Total score: 16.0 (accuracy: 4.0, completeness: 4.0, fluency: 4.0, conciseness: 4.0)
--
=====
Dataset: housing_3.json
Few-shot n: 1
Total score: 15.75 (accuracy: 4.0, completeness: 3.8333333333333335, fluency: 3.9166666666666665, conciseness: 4.0)
--
Few-shot n: 3
Total score: 15.75 (accuracy: 4.0, completeness: 3.8333333333333335, fluency: 3.9166666666666665, conciseness: 4.0)
--
Few-shot n: 5
Total score: 15.916666666666666 (accuracy: 4.0, completeness: 4.0, fluency: 3.9166666666666665, conciseness: 4.0)
--
=====
Dataset: mushroom_1.json
Few-shot n: 1

In [8]:
llm.inspect_history(n=1)




You are helping users understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative.
---
Follow the following format
Context: what the model predicts
Explanation: explanation of the model's prediction
Explanation Format: format the explanation is given in
Narrative: human-readable narrative version of the explanation
---
Example 1
Context: The model predicts whether a student will pass their class
Explanation: (Family eductional support, yes, 1.21), (In a romantic relationship, no, 1.12), (Age, 17, -0.45)
Explanation Format: SHAP feature contribution in (feature_name, feature_value, contribution) format
Narrative: The lack of a romantic relationship and having family support suggest the student is more likely to pass. However, being 17 indicates a lower probability of passing.
Example 2
Context: The model predicts whether a student will pass their class
Explanation: (In a romantic relationship, yes,

"\n\n\nYou are helping users understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative.\n---\nFollow the following format\nContext: what the model predicts\nExplanation: explanation of the model's prediction\nExplanation Format: format the explanation is given in\nNarrative: human-readable narrative version of the explanation\n---\nExample 1\nContext: The model predicts whether a student will pass their class\nExplanation: (Family eductional support, yes, 1.21), (In a romantic relationship, no, 1.12), (Age, 17, -0.45)\nExplanation Format: SHAP feature contribution in (feature_name, feature_value, contribution) format\nNarrative: The lack of a romantic relationship and having family support suggest the student is more likely to pass. However, being 17 indicates a lower probability of passing.\nExample 2\nContext: The model predicts whether a student will pass their class\nExplanation: (In a romantic

## Bootstrapped few-shot
Next, we repeat the experiment with the addition of 3 examples bootstrapped by DSPy to optimize the evaluation metrics.

In [11]:
for dataset in runners:
    runner = runners[dataset]
    print(f"Dataset: {dataset}")
    for i, j in [[0, 1], [0, 3], [3, 3]]:
        print(f"Few-shot n: {i}, Bootstrapped n: {j}")
        scores = runner.run_bootstrap_few_shot_experiment(n_labeled_few_shot=i, n_bootstrapped_few_shot=j)
        update_results("bootstrap_few_shot", dataset, scores, {"n_few_shot": i, "n_bootstrapped_few_shot": j, "prompt": prompts[0]})
        pretty_print(scores)
        print("--")
    results_df = pd.DataFrame(results)
    results_df.to_csv(f"results_{i}_{j}.csv")
    print("=====")

Dataset: housing_1.json
Few-shot n: 0, Bootstrapped n: 1


 30%|███       | 3/10 [00:00<00:00, 214.08it/s]


Bootstrapped 1 full traces after 4 examples in round 0.


 30%|███       | 3/10 [00:00<00:00, 206.66it/s]


Bootstrapped 1 full traces after 4 examples in round 0.


...

Bootstrapped 3 full traces after 5 examples in round 0.


 40%|████      | 4/10 [00:00<00:00, 104.29it/s]


Bootstrapped 3 full traces after 5 examples in round 0.


 40%|████      | 4/10 [00:00<00:00, 93.30it/s]


Bootstrapped 3 full traces after 5 examples in round 0.
Total score: 15.75 (accuracy: 3.8, completeness: 4.0, fluency: 3.95, conciseness: 4.0)
--
=====


In [12]:
result_df = pd.DataFrame(results)
result_df.to_csv("results.csv")
result_df


,dataset,prompt,total score,accuracy,completeness,fluency,conciseness,n_few_shot,n_bootstrapped_few_shot
0,housing_1.json,You are helping users understand an ML model's...,15.240000,3.52,4.000000,3.720000,4.0,5,NaN
1,housing_2.json,You are helping users understand an ML model's...,16.000000,4.00,4.000000,4.000000,4.0,1,NaN
2,housing_2.json,You are helping users understand an ML model's...,16.000000,4.00,4.000000,4.000000,4.0,3,NaN
3,housing_2.json,You are helping users understand an ML model's...,16.000000,4.00,4.000000,4.000000,4.0,5,NaN
4,housing_3.json,You are helping users understand an ML model's...,15.750000,4.00,3.833333,3.916667,4.0,1,NaN
5,housing_3.json,You are helping users understand an ML model's...,15.750000,4.00,3.833333,3.916667,4.0,3,NaN
6,housing_3.json,You are helping users understand an ML model's...,15.916667,4.00,4.000000,3.916667,4.0,5,NaN
7,mushroom_1.json,You are helping users understand an ML model's...,14.050000,2.80,3.800000,3.450000,4.0,1,NaN
8,mushroom_1.json,You are helping users understand an ML model's...,14.500000,3.40,3.600000,3.500000,4.0,3,NaN
9,mushroom_1.json,You are helping users understand an ML model's...,13.950000,2.60,3.800000,3.550000,4.0,5,NaN
